[Professor GitHub Repository](https://github.com/bardhprenkaj/ML_labs/blob/main/resources/project/project_description.pdf)

[Project GitHub Repository](https://github.com/Naxetee/ML_FinalProject)

# Index

- **Python Libraries**
- **Task 1**
    - *Reading Datasets*
    - *Exercise 1*
    - *Exercise 2*
    - *Exercise 3*
    - *Exercise 4*
    - *Exercise 5*


# Python Libraries


In [41]:
import pandas as pd
import numpy as np
import datetime as dt

# Task 1

During this first task we will apply some **pre-processing** operations to our raw data. So we will directly modify the data in *data/dataset/*, so that the data in *data/dataset_fixed/* will always remain unmodified.

## Reading the Datasets

At first, let's read the **.csv** files using a *pandas* function called **read_csv** whose parameters are:
- *filepath*: the path of the .csv files.
- *header*: will be always 0 to skip the columns' name row.
- *names*: gives a name to each column of the table.


In [42]:
data_path = "../../data/dataset/sample"

ap_fixed = pd.read_csv(data_path+'/anagraficapazientiattivi.csv', header=0 ,names=['idcentro','idana','sesso','annodiagnosidiabete','tipodiabete','scolarita','statocivile','professione','origine','annonascita','annoprimoaccesso','annodecesso'])
diag_fixed = pd.read_csv(data_path+'/diagnosi.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
elp_fixed = pd.read_csv(data_path+'/esamilaboratorioparametri.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
ei_fixed = pd.read_csv(data_path+'/esamistrumentali.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
pdf_fixed = pd.read_csv(data_path+'/prescrizionidiabetefarmaci.csv', header=0 ,names=['idcentro','idana','data','codiceatc','quantita','idpasto','descrizionefarmaco'])
pdnf_fixed = pd.read_csv(data_path+'/prescrizionidiabetenonfarmaci.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
pnd_fixed = pd.read_csv(data_path+'/prescrizioninondiabete.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])

In [43]:
ap = ap_fixed.copy(deep=True)
diag = diag_fixed.copy(deep=True)
elp = elp_fixed.copy(deep=True)
ei = ei_fixed.copy(deep=True)
pdf = pdf_fixed.copy(deep=True)
pdnf = pdnf_fixed.copy(deep=True)
pnd = pnd_fixed.copy(deep=True)

In [44]:
# Display example observations
print(ap.shape)
ap.head()

(250000, 12)


,idcentro,idana,sesso,annodiagnosidiabete,tipodiabete,scolarita,statocivile,professione,origine,annonascita,annoprimoaccesso,annodecesso
0,234,5697,M,2009.0,5,NaN,NaN,NaN,NaN,1949,2011.0,NaN
1,66,3539,M,2011.0,5,NaN,NaN,NaN,NaN,1933,2012.0,2015.0
2,100,3080,M,1988.0,5,NaN,NaN,NaN,NaN,1936,1991.0,NaN
3,112,3261,F,2014.0,5,NaN,NaN,NaN,NaN,1945,2014.0,NaN
4,160,9460,F,2014.0,5,NaN,NaN,NaN,NaN,1970,2016.0,NaN


In [45]:
print(diag.shape)
diag.head()

(4978504, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,1980-01-01,AMD247,491.20
1,1,5,1986-01-01,AMD247,272.0
2,1,5,1987-01-01,AMD083,NaN
3,1,5,1987-01-01,AMD247,401
4,1,5,1997-12-01,AMD247,410


In [46]:
print(elp.shape)
elp.head()

(29398377, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,2005-01-18,AMD001,169.0
1,1,5,2005-01-18,AMD002,76.0
2,1,5,2005-01-18,AMD004,135.0
3,1,5,2005-01-18,AMD005,80.0
4,1,5,2005-06-06,AMD007,169.0


In [47]:
print(ei.shape)
ei.head()

(1062856, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,2006-01-04,AMD051,N
1,1,5,2006-11-14,AMD041,P
2,1,5,2006-11-20,AMD040,P
3,1,5,2007-06-01,AMD040,N
4,1,5,2008-01-16,AMD040,N


In [48]:
print(pdf.shape)
pdf.head()

(7289926, 7)


,idcentro,idana,data,codiceatc,quantita,idpasto,descrizionefarmaco
0,82,782,2008-04-02,A10BA02,1.0,1,GLUCOPHAGE*60CPR RIV 1000MG
1,82,782,2008-04-02,A10BA02,1.0,3,GLUCOPHAGE*60CPR RIV 1000MG
2,82,782,2008-04-02,A10BA02,1.0,5,GLUCOPHAGE*60CPR RIV 1000MG
3,82,782,2008-04-02,A10BB09,3.0,1,DIAMICRON*60CPR R.M. 30MG
4,82,782,2008-05-02,A10BA02,1.0,1,GLUCOPHAGE*60CPR RIV 1000MG


In [49]:
print(pdnf.shape)
pdnf.head()

(569353, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,2008-06-20,AMD152,NaN
1,1,5,2013-08-27,AMD152,NaN
2,1,5,2013-12-31,AMD086,S
3,1,5,2013-12-31,AMD228,S
4,1,34,2005-01-07,AMD090,S


In [50]:
print(pnd.shape)
pnd.head()

(5573033, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,2005-01-18,AMD121,C09AA05
1,1,5,2005-01-18,AMD124,C10AA05
2,1,5,2005-01-18,AMD124,C10AX06
3,1,5,2005-06-21,AMD121,C09AA05
4,1,5,2005-06-21,AMD124,C10AA05


## Exercise 1

**Select Events of Interest -**
We want only patients with at leats one cardiovascular event in their trajectories. These events have the folloging codes:
- **AMD047**: Myocardial infarction
- **AMD048**: Coronary angioplasty
- **AMD049**: Coronary bypass
- **AMD071**: Ictus
- **AMD081**: Lower limb angioplasty
- **AMD082**: Peripheral By-pass Lower Limbs
- **AMD208**: Revascularization of intracranial and neck vessels
- **AMD303**: Ischemic stroke

Now let's try to filter all the tables containing the parameter **codiceamd** so as to select just the rows describing one cardio-vascular event.


In [51]:
codes = ['AMD047', 'AMD048', 'AMD049', 'AMD071', 'AMD081', 'AMD082', 'AMD208', 'AMD303']

def selectEvents(df:pd.DataFrame, events_codes:list) -> pd.DataFrame:
    return df[df.codiceamd.isin(events_codes)]

## pnd_cardEvents = selectEvents(pnd,codes)

## pdnf_cardEvents = selectEvents(pdnf,codes)

## ei_cardEvents = selectEvents(ei,codes)

## elp_cardEvents = selectEvents(elp,codes)

diag_cardEvents = selectEvents(diag,codes)
print("BEFORE pre-processing: ", diag.shape)
print("AFTER  pre-processing: ", diag_cardEvents.shape)
diag_cardEvents.head()


BEFORE pre-processing:  (4978504, 5)
AFTER  pre-processing:  (328478, 5)


,idcentro,idana,data,codiceamd,valore
7,1,5,2000-12-02,AMD049,S
9,1,5,2001-03-22,AMD049,S
13,1,5,2002-02-11,AMD049,S
15,1,5,2002-02-14,AMD049,S
17,1,5,2003-01-17,AMD049,S


We realise that the only table with these cardio-vascular events is **diagnosi**, so we create a new DataFrame with all the rows that has one of these events, and we call it: **diag_cardEvents**. And from this table, we extract the id's of the patients that have suffered at least one cardio-vascular event during his life:

In [52]:
patientsOfInterest = diag_cardEvents.filter(['idcentro','idana']).drop_duplicates()
print("Number of patients with at least one cardio-vascular event: ", patientsOfInterest.shape[0], "/", ap.shape[0] , "patients.")

Number of patients with at least one cardio-vascular event:  70193 / 250000 patients.


Finally, we have to delete from the rest of the tables all the information which is not associated to the remaining patients:

In [53]:
ap = pd.merge(ap, patientsOfInterest, on=['idcentro','idana'], how='right')[ap.columns]
diag = pd.merge(diag, patientsOfInterest, on=['idcentro','idana'], how='right')[diag.columns]
elp = pd.merge(elp, patientsOfInterest, on=['idcentro','idana'], how='right')[elp.columns]
ei = pd.merge(ei, patientsOfInterest, on=['idcentro','idana'], how='right')[ei.columns]
pdf = pd.merge(pdf, patientsOfInterest, on=['idcentro','idana'], how='right')[pdf.columns]
pdnf = pd.merge(pdnf, patientsOfInterest, on=['idcentro','idana'], how='right')[pdnf.columns]
pnd = pd.merge(pnd, patientsOfInterest, on=['idcentro','idana'], how='right')[pnd.columns]

## Exercise 2

Borrar pacientes que:
  - fecha nacimiento > fecha primo accessso || fecha nacimiento > fecha muerte || fecha nacimiento > fecha diagnostico
  - fecha primo accesso > fecha muerte || fecha primo accesso > fecha diagnostico
  - fecha diagnostico > fecha muerte

Resto tablas, borrar pacientes que:
  - fecha < fecha nacimiento || fecha < fecha primo accesso || fecha > fecha muerte

En tablas de medicamentos:
  - fecha entre fecha diagnostico y fecha muerte

First we want to select from our tables the patients with dates that are valid (with the birth year smaller than the death year, the birth year smaller than the year of first access and so on)

In [54]:
ap.shape

(70193, 12)

In [55]:
ap = ap[(ap['annonascita'].notna()) & (ap['annoprimoaccesso'].notna()) & (ap['annodiagnosidiabete'].notna()) & (ap['annonascita']<=ap['annodecesso']) & (ap['annoprimoaccesso']<=ap['annodecesso']) & (ap['annodiagnosidiabete']<=ap['annodecesso'])]
print(ap.shape)

ap = pd.concat([ap, ap[ap['annodecesso'].isna()]])
print(ap.shape)

ap = ap[(ap['annonascita']<=ap['annodiagnosidiabete']) & (ap['annonascita']<=ap['annoprimoaccesso'])]
print(ap.shape)

(6745, 12)
(6745, 12)
(6745, 12)


In [56]:
ap_test = ap.filter(items=['idcentro', 'idana'])
print(ap_test.shape)
ap_test.drop_duplicates()
print(ap_test.shape)

(6745, 2)
(6745, 2)


In [59]:
def exercise2(df:pd.DataFrame, test:pd.DataFrame) -> pd.DataFrame:
    # First we eliminate the patients that are no longer in the first table and the dates that do not exist
    df = df.merge(test, on=['idcentro', 'idana'])
    df = df[df['data'].notna()]

    # We assure that the dates are valid and that the year is first
    df['data'] = pd.to_datetime(df['data'], yearfirst=True)

    # Now we create a new table with the dates from the first and the second table
    ap_filter = ap.filter(items=['idcentro', 'idana', 'annonascita', 'annoprimoaccesso', 'annodecesso'])
    df = df.merge(ap_filter, on=['idcentro', 'idana'])

    # Now we take the rows of this table that have valid dates
    df = df[df['data'].dt.strftime("%Y").astype(float) <= df['annodecesso']]
    df = pd.concat([df, ap_filter[ap_filter['annodecesso'].isna()]])
    df = df[(df['data'].dt.strftime("%Y").astype(float) >= df['annoprimoaccesso']) & (df['data'].dt.strftime("%Y").astype(float) >= df['annonascita'])]
    if 'annodiagnosidiabete' in df.columns:
        df = df[(df['data'].dt.strftime("%Y").astype(float) >= df['annodiagnosidiabete'])]

    # Now we take into account only the new diagnosis but we need to delete the extra columns
    return df.filter(items=df.columns)

In [60]:
aux = diag.shape[0]
diag = exercise2(diag,ap_test)
print(f"Diag: {aux} => {diag.shape[0]}")

Diag: 2732801 => 276585


In [61]:
aux = elp.shape[0]
elp = exercise2(elp,ap_test)
print(f"Elp: {aux} => {elp.shape[0]}")

Elp: 10373351 => 957438


In [62]:
aux = ei.shape[0]
ei = exercise2(ei,ap_test)
print(f"Ei: {aux} => {ei.shape[0]}")

Ei: 425604 => 35024


In [63]:
aux = pdf.shape[0]
pdf = exercise2(pdf,ap_test)
print(f"Pdf: {aux} => {pdf.shape[0]}")

Pdf: 2797426 => 269148


In [64]:
aux = pdnf.shape[0]
pdnf = exercise2(pdnf,ap_test)
print(f"Pdnf: {aux} => {pdnf.shape[0]}")

Pdnf: 242508 => 23834


In [65]:
aux = pnd.shape[0]
pnd = exercise2(pnd,ap_test)
print(f"Pnd: {aux} => {pnd.shape[0]}")

Pnd: 2801883 => 271339


## Exercise 4

**Modify the actual ranges of** ***esamilaboratorioparameteri*** **-** see this Table:

| Code | Descriptive Name | True Range |
|----------|----------|----------|
| AMD004 | Systolic blood pressure | 40 <= x <= 200 |
| AMD005 | Diastolic blood pressure | 40 <= x <= 130 |
| AMD007 | Fasting blood glucose | 50 <= x <= 500 |
| AMD008 | HbAlc | 5 <= x <= 15 |
| AMD009 | Creatininemia | Not Available |
| AMD111 | Microalbuminuria | Not Available |
| STITCH001 | BMI | Not Available |
| STITCH002 | LDL Choresterlo | 30 <= x <= 30|
| STITCH003 | Non-HDL Cholesterlo | 60 <= x <= 330 |
| STITCH004 | eGFR MDRD | Not Available |
| STITCH005 | eGFR CKD-EPI | Not Available |

What we are doing is to use a **MinMaxScaler** from *sklearn* library so as to make the values described in the upper table fit into the true ranges. For the **NAN** ranges, we do nothing.

In [66]:
labels = ['AMD004', 'AMD005', 'AMD007', 'AMD008', 'AMD009', 'AMD111', 'STITCH001', 'STITCH002', 'STITCH003', 'STITCH004', 'STITCH005']

print("BEFORE RE-SCALING:\n")
print("    AMD    Min  /  Max")
print("========================")

for label in labels:
    aux = elp[elp.codiceamd == label].filter(['valore']).values.tolist()
    if len(aux) > 0:
        print(f"{label}:    {np.min(aux):.1f} / {np.max(aux):.1f}")
    else:
        print(f"{label}:  is null")

BEFORE RE-SCALING:

    AMD    Min  /  Max
AMD004:    0.0 / 701.0
AMD005:    0.0 / 980.0
AMD007:    0.0 / 978.0
AMD008:    0.0 / 98.0
AMD009:    nan / nan
AMD111:    -1.0 / 8950.0
STITCH001:  is null
STITCH002:  is null
STITCH003:  is null
STITCH004:  is null
STITCH005:  is null


In [67]:
from sklearn.preprocessing import MinMaxScaler

NAN = np.nan
newRanges = [['AMD004', 'AMD005', 'AMD007', 'AMD008', 'AMD009', 'AMD111', 'STITCH001', 'STITCH002', 'STITCH003', 'STITCH004', 'STITCH005'],
            [40.0,40.0,50.0,5.0,NAN,NAN,NAN,30.0,60.0,NAN,NAN],
            [200.0,130.0,500.0,15.0,NAN,NAN,NAN,300.0,330.0,NAN,NAN]]

def scaling(df:pd.DataFrame, label:str, minValue:float, maxValue:float) -> pd.DataFrame:
    if (minValue != NAN) and (maxValue != NAN) and (df[df.codiceamd == label].shape[0]>0):
        aux = df[df.codiceamd == label].copy()
        min_max_scaler = MinMaxScaler(feature_range=(minValue,maxValue))
        aux[['valore']] = min_max_scaler.fit_transform(aux[['valore']])
        df[df.codiceamd == label] = aux
    return df


# Let's apply the function to a new DataFrame copied from elp_new
for k in range(len(newRanges[0])):
    elp = scaling(elp, newRanges[0][k], newRanges[1][k], newRanges[2][k])

Now we ensure manually that it has worked by printing the maximum and minimum values for each AMD Code

In [68]:
print("\n\nAFTER RE-SCALING:\n")
print("    AMD    Min  /  Max")
print("========================")

for label in labels:
    aux = elp[elp.codiceamd == label].filter(['valore']).values.tolist()
    if len(aux) > 0:
        print(f"{label}:    {np.min(aux):.1f} / {np.max(aux):.1f}")
    else:
        print(f"{label}:  is null")




AFTER RE-SCALING:

    AMD    Min  /  Max
AMD004:    40.0 / 200.0
AMD005:    40.0 / 130.0
AMD007:    50.0 / 500.0
AMD008:    5.0 / 15.0
AMD009:    nan / nan
AMD111:    nan / nan
STITCH001:  is null
STITCH002:  is null
STITCH003:  is null
STITCH004:  is null
STITCH005:  is null


So it matches with the table given by the exercise

## Exercise 5

**Cohort selection and label definition** - use only those patients that after all the previous steps contain at least two events before calculating the label. Let $\mathcal{P}=\lbrace p_1, \cdots,p_n\rbrace$ be the set of all patients in the dataset. Let $d(e^i_k)$ be the date of the last event $e_k$ for patient $p_i\in\mathcal{P}$. The label of the patient $p_i$ is calculated as follows:

$y(p_i)=\begin{dcases}
                1 & \text{if, within }d(e^i_k)-6 \text{ months, }p_i\text{ has a cardiovascular event}\\
                0 & \text{otherwise}
        \end{dcases}$

Eliminate the patients that have a trajectory shorten or equal to 6 months.


We first create a DataFrame that contains all the patients and their cardiovascular events, each one with its date (it's already computed as **diag_cardEvents**):

In [69]:
diag_cardEvents.head()

,idcentro,idana,data,codiceamd,valore
7,1,5,2000-12-02,AMD049,S
9,1,5,2001-03-22,AMD049,S
13,1,5,2002-02-11,AMD049,S
15,1,5,2002-02-14,AMD049,S
17,1,5,2003-01-17,AMD049,S


Now for each patient of the frame **diag_cardEvents**, we are going to take its two last cardiovascular events dates, so as to check if they happened with less than 6 months of difference. For this, we define the function **labeling**

In [70]:
def labeling(df:pd.DataFrame) -> pd.DataFrame:

    # First we define a dataframe that contains the ids and the dates of every cardiovascular event, sorted by descending date
    cardEvents_sorted = df.sort_values('data',ascending=False)

    # Now we create a new dataset called classes, that represents the result of the function y(p_i) defined previously:
    classes = pd.DataFrame(columns=['idcentro','idana','class'])

    # Now for each patient (idcentro,idana), we check if it verify the condition explained previously:
    for label, subDataframe in cardEvents_sorted.groupby(['idcentro','idana']):

        dates = subDataframe['data'].dropna().values

        # We ignore those patients who have just one event
        if len(dates) > 1:

            # We extract the first and the 2 lastest dates and convert them into datetime objects
            minDate = dates[-1]
            maxDates = dates[0:2]
            minDate = dt.date.fromisoformat(minDate)
            maxDates = dt.date.fromisoformat(maxDates[0]), dt.date.fromisoformat(maxDates[1])

            # We ignore patients with a trajectory shorter than 6 months (6*30 days)
            if maxDates[0]-minDate > dt.timedelta(days=6*30):

                # We calculate the difference between the latest 2 dates and compare it with 6 months
                c = maxDates[0]-maxDates[1] < dt.timedelta(days=6*30)

                # We add a new column with the label and the class of the patient
                classes.loc[classes.count()[0]] = [label[0], label[1], int(c)]

    return classes




Now, in the **classes** dataframe, we have labeled the patients that have more than one event and a trajectory larger than 6 months

In [71]:
labels = labeling(diag_cardEvents)

In [72]:
print(f"We still have: {labels.shape[0]}/{patientsOfInterest.shape[0]} patients")

We still have: 44550/70193 patients


So we do an inner merge operation again with every dataframe to take just the patients of the frame **classes**

In [73]:
ap = pd.merge(ap, labels, on=['idcentro','idana'], how='right')[ap_fixed.columns]
diag = pd.merge(diag, labels, on=['idcentro','idana'], how='right')[diag_fixed.columns]
elp = pd.merge(elp, labels, on=['idcentro','idana'], how='right')[elp_fixed.columns]
ei = pd.merge(ei, labels, on=['idcentro','idana'], how='right')[ei_fixed.columns]
pdf = pd.merge(pdf, labels, on=['idcentro','idana'], how='right')[pdf_fixed.columns]
pdnf = pd.merge(pdnf, labels, on=['idcentro','idana'], how='right')[pdnf_fixed.columns]
pnd = pd.merge(pnd, labels, on=['idcentro','idana'], how='right')[pnd_fixed.columns]
patientsOfInterest_labeled = pd.merge(patientsOfInterest, labels, on=['idcentro','idana'], how='right')
